In [1]:
from flask import Flask, render_template, url_for, request, flash, redirect, jsonify 
from flask_sqlalchemy import SQLAlchemy
import json
import os
from datetime import datetime
from random import randint

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///app.db'
db = SQLAlchemy(app)


# Database Model
class blogInfo(db.Model):
    id = db.Column(db.Integer, primary_key = True, default = randint(0,100000))
    title = db.Column(db.String(), nullable = False)
    content = db.Column(db.String(200), nullable = False)
    date_uploaded = db.Column(db.DateTime, default = datetime.utcnow)
    
    def __repr__(self):
        return '<Task %r>' % self.id
    
    @property
    def serialize(self):
        return{
            'id': self.id,
            'title': self.title,
            'content': self.content,
            'date_uploaded': self.date_uploaded,
        }


@app.route("/post_confirmation")
def post_confirmation():
    return 'Post was created! Returning to admin page now.'

#code for retrieving blog--every blog
@app.route('/retr_blogs', methods =["GET"])
def retrieve_blogs():
    blog = blogInfo.query.all()
    serialized_data = []

#-- retrieving blog by a single id
@app.route('/retr_blog/<int:id>', methods=["GET"])
def retr_single_blog(id):
    blog = blogInfo.filter_by(id=id).first()
    serialized_blog = blog.serialize


#Home page
@app.route('/')
def home_page():
    with open('posts.json', 'r') as file:
        posts = json.load(file)["posts"]

    return render_template('index.html',posts_list = posts)

# Admin interface, and adding a blog post to the database
@app.route('/admin', methods=('GET','POST'))
def admin():
    if request.method == 'POST':
        post_title = request.form['title']
        post_content = request.form['content']
        new_blog=blogInfo(title=post_title,content=post_content)
        db.session.add(new_blog)
        db.session.commit()
        # blog_id = getattr(new_blog, "id")

        return redirect('/post_confirmation')
    else:
        return render_template('admin.html')




# Error Code Handling
@app.errorhandler(404)
def not_found(e):
    return render_template('404.html'), 404

def main():
    if not os.path.exists('app.db'):
        with app.app_context():
            db.create_all()
    app.run()


if __name__ == "__main__":
    main()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [31/Mar/2023 15:51:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [31/Mar/2023 15:51:35] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [31/Mar/2023 15:51:40] "GET /admin HTTP/1.1" 200 -
127.0.0.1 - - [31/Mar/2023 15:51:43] "POST /admin HTTP/1.1" 302 -
127.0.0.1 - - [31/Mar/2023 15:51:43] "GET /post_confirmation HTTP/1.1" 200 -
